In [14]:
from random import shuffle
import numpy as np
import h5py
import cv2
import glob
import matplotlib.pyplot as plt
import cv
import time
import os
shuffle_data = True  # shuffle the addresses before saving
hdf5_path = '../train/dataset.hdf5'  # address to where you want to save the hdf5 file
cat_dog_train_path = 'imagenet_data/val_raw/*'
# read addresses and labels from the 'train' folder
addrs = glob.glob(cat_dog_train_path)
shuffle_data = True

In [16]:
addrs = [x for x in addrs if x[-9] == 'n']
print len(addrs), addrs[0]

1000 imagenet_data/val_raw/n03868242


In [23]:
#for i,x in enumerate(addrs):
#    src = '/datasets/imagenet/' + x[:4] + '/' + x
#    dst = '../train/' + x
#    os.symlink(src, dst)

In [3]:
classes = addrs[:]
classes.sort()
class_to_idx = {classes[i]: i for i in range(len(classes))}

In [4]:
sum = [0]
X_train_addrs = []
Y_train_labels = []
for i,x in enumerate(addrs):
    cat_dog_train_path = x + '/*'
    # read addresses and labels from the 'train' folder
    temp = glob.glob(cat_dog_train_path)
    sum[0] = sum[0] + len(temp)
    X_train_addrs = X_train_addrs + temp
    Y_train_labels = Y_train_labels + list(np.full((len(temp)), class_to_idx[x]))
    if i % 100 == 0:
        print i,x, sum[0], len(X_train_addrs)

0 n03868242 1186 1186
100 n03032252 131633 131633
200 n02795169 264446 264446
300 n07579787 393107 393107
400 n03777754 522736 522736
500 n02093991 650757 650757
600 n04200800 777304 777304
700 n03692522 902836 902836
800 n03131574 1030262 1030262
900 n04118776 1158361 1158361


In [5]:
#Shuffle data

print Y_train_labels[:10]

if shuffle_data:
    c = list(zip(X_train_addrs, Y_train_labels))
    shuffle(c)
    shuffle(c)
    shuffle(c)
    X_train_addrs, Y_train_labels = zip(*c)
    
print Y_train_labels[:10]

[690, 690, 690, 690, 690, 690, 690, 690, 690, 690]
(591, 141, 813, 777, 42, 422, 280, 552, 707, 231)


In [7]:
data_order = 'tf'  # 'th' for Theano, 'tf' for Tensorflow
# check the order of data and chose proper data shape to save images
if data_order == 'th':
    train_shape = (len(X_train_addrs), 3, 224, 224)
elif data_order == 'tf':
    train_shape = (len(X_train_addrs), 224, 224, 3)
# open a hdf5 file and create earrays
hdf5_file = h5py.File(hdf5_path, mode='w')
hdf5_file.create_dataset("train_img", train_shape, np.float32)
hdf5_file.create_dataset("train_mean", (3,), np.float32)
hdf5_file.create_dataset("train_std", (3,), np.float32)
hdf5_file.create_dataset("train_labels", (len(Y_train_labels),), np.int8)
hdf5_file["train_labels"][...] = Y_train_labels

IOError: Unable to create file (unable to truncate a file which is already open)

In [18]:
mean = np.zeros((3,), np.float32)
print mean.shape
# loop over train addresse
start = time.time()
for i in range(len(X_train_addrs)):
    # print how many images are saved every 1000 images
    if i % 1000 == 0 and i > 1:
        print 'Train data: {}/{}'.format(i, len(X_train_addrs))
        print time.time() - start
        start = time.time()
        if i % 10000 == 0 :
            break
    # read an image and resize to (224, 224)
    # cv2 load images as BGR, convert it to RGB
    addr = X_train_addrs[i]
    img = cv2.imread(addr)
    img = cv2.resize(img, (224, 224), interpolation=cv2.INTER_CUBIC)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    #M = np.sqrt(((img.shape[0]/2.0)**2.0)+((img.shape[1]/2.0)**2.0))
    #M = img.shape[1]/np.log(M)
    #cv.LogPolar(cv.fromarray(img), cv.fromarray(polar_image),(img.shape[0]/2, img.shape[1]/2), M, cv.CV_INTER_LINEAR+cv.CV_WARP_FILL_OUTLIERS)

    # add any image pre-processing here
    # if the data order is Theano, axis orders should change
    if data_order == 'th':
        img = np.rollaxis(img, 2)
    # save the image and calculate the mean so far
    hdf5_file["train_img"][i, ...] = img[None]
    mean[0] += np.mean(img[:,:,0]) / float(len(Y_train_labels))
    mean[1] += np.mean(img[:,:,1]) / float(len(Y_train_labels))
    mean[2] += np.mean(img[:,:,2]) / float(len(Y_train_labels))


(3,)
Train data: 1000/1290129
107.424635172
Train data: 2000/1290129
114.413421154
Train data: 3000/1290129
114.30145812
Train data: 4000/1290129
99.6844809055
Train data: 5000/1290129
97.1008198261
Train data: 6000/1290129
97.1042850018
Train data: 7000/1290129
108.806693077


KeyboardInterrupt: 

In [ ]:
# loop over validation addresses
for i in range(len(val_addrs)):
    # print how many images are saved every 1000 images
    if i % 1000 == 0 and i > 1:
        print 'Validation data: {}/{}'.format(i, len(val_addrs))
    # read an image and resize to (224, 224)
    # cv2 load images as BGR, convert it to RGB
    addr = val_addrs[i]
    img = cv2.imread(addr)
    img = cv2.resize(img, (224, 224), interpolation=cv2.INTER_CUBIC)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # add any image pre-processing here
    # if the data order is Theano, axis orders should change
    if data_order == 'th':
        img = np.rollaxis(img, 2)
    # save the image
    hdf5_file["val_img"][i, ...] = img[None]

In [17]:
#validation 256 * 256 resize code

In [34]:
sum = [0]
for i,x in enumerate(addrs):
    cat_dog_train_path = x + '/*'
    # read addresses and labels from the 'train' folder
    temp = glob.glob(cat_dog_train_path)
    sum[0] = sum[0] + len(temp)
    print sum[0]
    for y in temp:
        #X_train_addrs = X_train_addrs + temp
        #Y_train_labels = Y_train_labels + list(np.full((len(temp)), class_to_idx[x]))
        #print y.split('/')[-2], '../val/'+ x + '/' + y.split('/')[-1]
        #break
        #print 'imagenet_data/val/'+ y.split('/')[-2] + '/' + y.split('/')[-1]
        img = cv2.imread(y)
        #plt.imshow(img)
        #plt.show()
        img = cv2.resize(img, (256, 256), interpolation=cv2.INTER_CUBIC)
        cv2.imwrite('imagenet_data/val/'+ y.split('/')[-2] + '/' + y.split('/')[-1],img)


50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500


KeyboardInterrupt: 